In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="k", fc=(1, 0, 0, 0.5))
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

# Designs

In [ ]:
def chip_metadata(**kwargs):
    metadata = {}
    main_cell = chip(
        "my_chip", **{"metadata": metadata, **kwargs, "draw_trenches": False}
    )
    return metadata["my_chip"]

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
# chip_dims = np.array([23e3, 13e3])
chip_dims = np.array([40e3, 20e3])

In [ ]:
from snakenmake import *

metadata = {}
main_cell = chip(
    "Big Snake",
    snake,
    dims=chip_dims,
    split=8,
    feeding_channel_width=100,
    trench_spacing=2,
    lane_gap=20,
    trench_width=1.5,
    draw_trenches=False,
    metadata=metadata,
)
print(metadata)
plot_cell(main_cell)

In [ ]:
# S8 W1.3 L35 x2
# S8 W1.5 L35 x3
# S8 W1.5 L45 x2

# S8 W1.5 L35-65 sampler?

In [ ]:
%%time
from snakenmake import *

metadata = {}
params = [
    dict(split=4, fc=40, w=1.4, ts=1.6, lg=10),
    dict(split=4, fc=40, w=1.4, ts=1.1, lg=10),
    dict(split=4, fc=40, w=1.4, ts=1.1, lg=10),
    dict(split=8, fc=40, w=1.4, ts=1.6, lg=10),
    dict(split=8, fc=40, w=1.4, ts=1.1, lg=10),
    dict(split=8, fc=40, w=1.4, ts=1.1, lg=10),
]
chips = [
    chip(
        "Basilisk S{split} FC{fc} L35 W{w} TS{ts} LG{lg}".format(**params),
        snake,
        split=params["split"],
        feeding_channel_width=params["fc"],
        trench_spacing=params["ts"],
        lane_gap=params["lg"],
        tick_labels=False,
        ticks=True,
        tick_length=4,
        tick_margin=3,
        text=True,
        draw_trenches=True,
        metadata=metadata,
    )
    for params in params
]
# chips = [chips[0]]*6
main_cell = wafer(chips, "Basilisk \n BS/JQS 181115", text=True, mask=True)
write_gds(main_cell, "181116basilisk.gds")

In [ ]:
# print text summary for wafer spreadsheet

In [ ]:
plot_cell(main_cell)

In [ ]:
plot_cell(snakenmake.mask_alignment_cross())

In [ ]:
metadata

In [ ]:
plot_cell(chips[5])